In [10]:
from gpt_completer import GptCompleter
from IPython.display import display, Markdown
from pathlib import Path


SYSTEM_PROMPT_TEPMPLATE = """\
You are a programming assistant. 

When asked to update, please provide only the code that you want to add or change.


Source files are:
```
{filenames}
```

Source code:

{source_files}
"""


SOURCE_PART_TEMPLATE = """\
name: `{filename}`

contents:
```
{contents}
```
"""

INCLUDE_SUFFIXES = [".go", ".md"]
EXCLUDE_DIRS = [".git", "__pycache__", ".venv", "notebooks"]


def all_files_recursively(dir=".", exclude_dirs=EXCLUDE_DIRS):
    result = []
    for p in Path(dir).iterdir():
        if p.is_dir() and p.name in exclude_dirs:
            continue
        if p.is_file():
            result.append(p)
        else:
            result.extend(all_files_recursively(p, exclude_dirs))
    return result


def build_sys_prompt(dir=".."):
    all_files = all_files_recursively(dir)
    filenames = [str(p.relative_to(dir)) for p in all_files]
    parts = []
    for p in all_files:
        if p.suffix in INCLUDE_SUFFIXES:
            contents = p.read_text() or "EMPTY FILE"
            parts.append(
                SOURCE_PART_TEMPLATE.format(
                    filename=p.relative_to(dir), contents=contents
                )
            )
    return SYSTEM_PROMPT_TEPMPLATE.format(
        filenames="\n".join(filenames), source_files="\n".join(parts)
    )

In [ ]:
QUERY = """\
We are wrting a service in Go language.
You've been given with the initial project layout.
Please create a test for the case when Completer receives 429 Limit Reached error from the openai API.
"""
display(Markdown(GptCompleter(build_sys_prompt()).complete(QUERY)))